# 02 notebook

02_sampleデータから，いい感じのシフトを作成できるような構造を作成 & チューニングをする．

必要な処理

- シフト生成
- 従業員の希望を考慮する処理
- 人件費を計算
- 店舗のコメントを考慮する処理

- 最適なシフトであるかを評価する処理
    - 何かしらで定量化．
シフトの出力，生成

In [1]:
import os
import json
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

True

In [2]:
# key設定
api_key = os.getenv('GEMINI_API')
client = genai.Client(api_key=api_key)


# ファイル読み込み
with open('./../data/inputs/02_sample_prototype.json', 'r') as f:
    input_json = json.loads(f.read())

In [3]:
input = input_json

In [4]:
# systemprompt
system_prompt = """
あなたは、ユーザーからの多様なテキスト入力に対して、指定されたJSON形式のデータ構造を生成する、
高度に専門化されたAIアシスタントです。
あなたの最優先事項は、ユーザーの意図を正確に解釈し、指定されたスキーマに従って、
**厳密にJSON形式のデータのみを出力すること**です。
出力に余計な説明、前置き、後書き、または非JSON形式のテキストを一切含めないでください。
ユーザーの指示に不明瞭な点がある場合は、最も合理的な解釈に基づいて処理を進めてください。
\n\n## 出力形式の厳守
\n\n* 常に有効なJSONを生成してください。
\n* JSONのキーと値は、指定されたデータ型と形式に厳密に従ってください。
\n* 数値、日付、時刻などのデータは、指定されたフォーマットで出力してください。
\n\n## 日付と時刻の処理に関する指示\n\n日付や時刻に関する情報がテキストに含まれる場合、
適切な日付（YYYY-MM-DD）や時刻（HH:MM）に変換してください。
特に「来週」などの相対的な表現は、コンテキストに基づいて絶対日付に変換してください。

あなたは、従業員のシフト要望とクルー評価を含むテキストから、
以下のJSONスキーマに従ってデータを抽出する専門家です。
汎用的な指示に加えて、以下の具体的なルールと例に厳密に従って作業してください。
\n\n## 出力JSONスキーマ\n\n```json\n{\n  \"shift_requests\": [\n    {\n      \"crew_id\": \"string\",\n      \"day\": \"YYYY-MM-DD\",\n      \"start_time\": \"HH:MM または '休み'\",\n      \"finish_time\": \"HH:MM または '休み'\"\n    }\n  ],\n  \"crew_evaluations\": [\n    {\n      \"crew_id\": \"string\",\n      \"text\": \"string\"\n    }\n  ]\n}\n```\n\n## 抽出ルール\n\n1.  **シフト要望の抽出**: \n
* `crew_id` は「crew_X」という表記から抽出します。\n
* **日付の変換**: 「MM月DD日」のような形式の日付は、コンテキストに基づいて日付に変換し、
`YYYY-MM-DD`形式で記述してください。\n
* **時刻の解釈と変換**: \n
* 「フルタイム」: 9時から18時\n
* 「午前中のみ」: 9時から13時\n
* 「午後から」: 13時から20時\n
* 「遅番」: 15時から22時\n
* 「通常通り」: 10時から19時\n
* 具体的な時間指定（例: 「9時から18時まで」）があれば、上記のデフォルトよりも優先してください。\n
* 「終日休み」と明記されている場合は、`start_time`と`finish_time`を共に「休み」と記述してください。\n
* **代替案の無視**: シフトの要望で複数の選択肢や代替案が提示された場合（例：「もし難しければ、〜も検討できます」）、
**最初に明確に述べられた要望のみを抽出し、代替案は無視してください。**\n\n2.  **クルー評価の抽出**: \n    * 「crew_Xの評価は、〜」というパターンを認識し、`crew_id`とそれに続く評価内容を`text`として抽出します。
"""

# モデル定義
response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=system_prompt
    ),
    contents=input,
)

ValidationError: 9 validation errors for _GenerateContentParameters
contents.list[union[Content,list[union[File,Part,str]],File,Part,str]]
  Input should be a valid list [type=list_type, input_value={'company': {'company_id'..., 'finish': '23:00'}}}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type
contents.Content.company
  Extra inputs are not permitted [type=extra_forbidden, input_value={'company_id': 1, 'open_t...せてください。'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden
contents.Content.worker
  Extra inputs are not permitted [type=extra_forbidden, input_value={'worker_1': {'user_id': ...', 'finish': '23:00'}}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden
contents.list[union[File,Part,str]]
  Input should be a valid list [type=list_type, input_value={'company': {'company_id'..., 'finish': '23:00'}}}}}, input_type=dict]
  

In [ ]:
print(response.text)

```json
{
  "shift_requests": [
    {
      "crew_id": "crew_1",
      "day": "2024-06-16",
      "start_time": "09:00",
      "finish_time": "18:00"
    },
    {
      "crew_id": "crew_1",
      "day": "2024-06-18",
      "start_time": "09:00",
      "finish_time": "18:00"
    },
    {
      "crew_id": "crew_1",
      "day": "2024-06-17",
      "start_time": "13:00",
      "finish_time": "20:00"
    },
    {
      "crew_id": "crew_1",
      "day": "2024-06-19",
      "start_time": "09:00",
      "finish_time": "13:00"
    },
    {
      "crew_id": "crew_1",
      "day": "2024-06-20",
      "start_time": "休み",
      "finish_time": "休み"
    },
    {
      "crew_id": "crew_2",
      "day": "2024-06-16",
      "start_time": "09:00",
      "finish_time": "13:00"
    },
    {
      "crew_id": "crew_2",
      "day": "2024-06-17",
      "start_time": "09:00",
      "finish_time": "13:00"
    },
    {
      "crew_id": "crew_2",
      "day": "2024-06-18",
      "start_time": "休み",
      "finish